In [1]:
import pandas as pd
import numpy as np
import warnings
import os
import datetime
import requests
import json

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
# CRED ACCESS TO GET ACCESS TOKEN
host = os.environ.get("DYNAMICS_HOST")
url = os.environ.get("DYNAMICS_URL")
creds_grand_type = os.environ.get("CREDS_GRAND_TYPE")
client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECREAT")

# Params
auth_params = {
    "grant_type": creds_grand_type,
    "client_id": client_id,
    "client_secret": client_secret,
    "resource": host,
}

# Request to get token access
response_token = requests.get(url=url, data=auth_params)
token = json.loads(response_token.text)["access_token"]

In [3]:
# End point for transfer order data
url = "transactions/transfer_orders_get"
end_point = os.environ.get("END_POINT") + url

# Heeader / params
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
}

# Var for get the datas``
start_date = "2024-07-01"
end_date = "2024-07-07"
pages = 1

In [13]:
df_transfer_order = pd.DataFrame()
df_info = pd.DataFrame()
has_more_page = True

while has_more_page:
    # Request to End Point for get transfer order data
    params = {
        "_r": {
            "DataAreaId": "MGL",  # mandatory option (MPR, MGL)
            "Page": pages,  # mandatory default first page 1
            "Limit": 50,  # mandatory default max limit 50
            "Sort": "ASC",  # mandatory default ASC option (ASC, DESC)
            "RequiredDate": "",
            "FromDate": start_date,  # mandatory format date (yyy-mm-dd) CreateDateTime
            "ToDate": end_date,  # mandatory format date (yyyy-mm-dd) CreateDateTime
            # "CustGroup":"Showroom", #option (Showroom)
            # "SalesStatus":"Invoiced"#option status ( OpenOrder, Delivered, Invoiced, Canceled )
        }
    }
    
    print(pages)

    response = requests.post(url=end_point, json=params, headers=headers)

    data_json = response.json().get("Data")

    # Normalize Page Data
    data = pd.json_normalize(data_json)

    total_pages = data['total_pages'].unique()
    
    df_info = pd.concat([df_info, data])

    # Normalize Header Data
    if (
        "Data" in data.columns
        and data["Data"].notna().any()
        and not data["Data"].apply(lambda x: x == []).all()
    ):
        df_header = pd.json_normalize(
            data=data[["current_page", "Data"]].explode("Data").to_dict(orient="records")
        )

    # Normalize Detail Data
    if (
        "Data.Details" in df_header.columns
        and df_header["Data.Details"].notna().any()
        and not df_header["Data.Details"].apply(lambda x: x == []).all()
    ):
        df_detail = pd.json_normalize(
            data=df_header[["Data.TransferId", "Data.Details"]]
            .explode("Data.Details")
            .to_dict(orient="records")
        )

    # Join header and detail
    data_transfer_order = df_header.drop(columns="Data.Details").merge(
        df_detail, "left", on="Data.TransferId", suffixes=("Header", "Detail")
    )
    
    df_transfer_order = pd.concat([df_transfer_order, data_transfer_order])

    if pages == 10:
        has_more_page = False
    else :
        pages = pages+1

1
2
3
4
5
6
7
8
9
10


In [14]:
df_info

,current_page,Data,per_page,total_pages,total_items
0,1,"[{'TransferId': 'MGL/TO/2302/28000015', 'Trans...",50,155,7749
0,2,"[{'TransferId': 'MGL/TO/2306/26000363', 'Trans...",50,155,7749
0,3,"[{'TransferId': 'MGL/TO/2312/07000033', 'Trans...",50,155,7749
0,4,"[{'TransferId': 'MGL/TO/2312/27000026', 'Trans...",50,155,7749
0,5,"[{'TransferId': 'MGL/TO/2401/00000012', 'Trans...",50,155,7749
0,6,"[{'TransferId': 'MGL/TO/2401/00000077', 'Trans...",50,155,7749
0,7,"[{'TransferId': 'MGL/TO/2401/00000137', 'Trans...",50,155,7749
0,8,"[{'TransferId': 'MGL/TO/2401/00000215', 'Trans...",50,155,7749
0,9,"[{'TransferId': 'MGL/TO/2401/00000467', 'Trans...",50,155,7749
0,10,"[{'TransferId': 'MGL/TO/2401/00000538', 'Trans...",50,155,7749


In [15]:
df_transfer_order

,current_page,Data.TransferId,Data.TransferStatus,Data.FromLocation,Data.FromLocationName,Data.ToLocation,Data.ToLocationName,Data.ShipDate,Data.ReceiveDate,Data.ShipDateActual,Data.ReceiveDateActual,Data.CancelledShipment,Data.AutoReceive,Data.VoucherShipment,Data.VoucherReceive,Data.ToType,Data.DistributionID,Data.Blister,Data.POSOrderId,Data.VendorEkspedisiId,Data.VendorEkspedisiName,Data.PackingCode,Data.POSCancel,Data.ReturnOrderId,Data.MarketPlaceId,Data.Notes,Data.AutoTransferOrder,Data.NoResiReturnEcommers,Data.PackingList,Data.OldTransferId,Data.TransportOrderId,Data.TransportUnitType,Data.TransportRouteRateId,Data.TransportTROType,Data.TransportSecondTRO,Data.TransportExpedisi,Data.TransportExpedisiName,Data.TransportStatus,Data.TransportPickupRequestDate,Data.TransportPickupConfirmDate,Data.TransportConfirmDate,Data.TransportDate,Data.TransportType,Data.TransportServiceType,Data.TransportServiceCategory,Data.TransportCityFrom,Data.TransportCityTo,Data.TransportRate,Data.TransportActualWeight,Data.TransportEstCost,Data.PickingRoute,Data.PickingBrand,Data.PickingJumlahKoli,Data.PickingBeratBarang,Data.PickingSuratJalanPOS,Data.PickingAWBResi,Data.PickingExpedision,Data.PickingJubelioStatus,Data.PickingShipperContanct,Data.PickingShipperContanctPhone,Data.PickingConsignContanct,Data.PickingConsignContanctPhone,Data.CreateDateTime,Data.ModifiedDateTime,Data.Details.ItemNumber,Data.Details.ItemName,Data.Details.Size,Data.Details.Color,Data.Details.Location,Data.Details.QtyTransfer,Data.Details.QtyShipped,Data.Details.QtyRemain,Data.Details.QtyReceived,Data.Details.ShipDate,Data.Details.ReceiveDate,Data.Details
0,1,MGL/TO/2302/28000015,Shipped,20114,OUTLET SZ ITC PERMATA HIJAU,20110,OUTLET SZ PEMALANG,2023-02-28T12:00:00,1900-01-01T12:00:00,2023-12-31T12:00:00,1900-01-01T12:00:00,No,0,MGL/TOV/2401/0000239,,AB,,0,,,,,No,,,,No,,No,,,,,,,,,,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,,,,,,0.0,0.0,0.0,,,0,0.00,,,,,,,,,2024-01-17T18:34:56Z,2024-07-04T18:08:30Z,2312000006981,Manzone Fameux Blue / Manzone Fameux Blue,S,,20114,1.0,1.0,1.0,0.0,2023-02-28T12:00:00,2023-12-21T12:00:00,NaN
1,1,MGL/TO/2302/28000015,Shipped,20114,OUTLET SZ ITC PERMATA HIJAU,20110,OUTLET SZ PEMALANG,2023-02-28T12:00:00,1900-01-01T12:00:00,2023-12-31T12:00:00,1900-01-01T12:00:00,No,0,MGL/TOV/2401/0000239,,AB,,0,,,,,No,,,,No,,No,,,,,,,,,,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,,,,,,0.0,0.0,0.0,,,0,0.00,,,,,,,,,2024-01-17T18:34:56Z,2024-07-04T18:08:30Z,2312000006547,Manzone Hopkin3 Blue / Manzone Hopkin3 Blue,S,,20114,1.0,1.0,1.0,0.0,2023-02-28T12:00:00,2023-12-21T12:00:00,NaN
2,1,MGL/TO/2302/28000015,Shipped,20114,OUTLET SZ ITC PERMATA HIJAU,20110,OUTLET SZ PEMALANG,2023-02-28T12:00:00,1900-01-01T12:00:00,2023-12-31T12:00:00,1900-01-01T12:00:00,No,0,MGL/TOV/2401/0000239,,AB,,0,,,,,No,,,,No,,No,,,,,,,,,,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,,,,,,0.0,0.0,0.0,,,0,0.00,,,,,,,,,2024-01-17T18:34:56Z,2024-07-04T18:08:30Z,2312000007464,Manzone Gronzy Navy / Manzone Gronzy Navy,S,,20114,1.0,1.0,1.0,0.0,2023-02-28T12:00:00,2023-12-21T12:00:00,NaN
3,1,MGL/TO/2302/28000015,Shipped,20114,OUTLET SZ ITC PERMATA HIJAU,20110,OUTLET SZ PEMALANG,2023-02-28T12:00:00,1900-01-01T12:00:00,2023-12-31T12:00:00,1900-01-01T12:00:00,No,0,MGL/TOV/2401/0000239,,AB,,0,,,,,No,,,,No,,No,,,,,,,,,,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,,,,,,0.0,0.0,0.0,,,0,0.00,,,,,,,,,2024-01-17T18:34:56Z,2024-07-04T18:08:30Z,2312000007960,Manzone Kent Grey_Light / Manzone Kent Grey_Light,S,,20114,1.0,1.0,1.0,0.0,2023-02-28T12:00:00,2023-12-21T12:00:00,NaN
4,1,MGL/TO/2302/28000015,Shipped,20114,OUTLET SZ ITC PERMATA HIJAU,20110,OUTLET SZ PEMALANG,2023-02-28T12:00:00,1900-01-01T12:00:00,2023-12-31T12:00:00,1900-01-01T12:00:00,No,0,MGL/TOV/2401/0000239,,AB,,0,,,,,No,,,,No,,No,,,,,,,,,,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01-01T12:00:00,1900-01